In [ ]:
encoded_obj = encode(obj)

if callable(key):
    key_func = key
    key = key_func(obj, encoded_obj)

if not callable(save_under_key):
    if hasattr(save_under_key, '__setitem__'):
        save_under_key[encoded_obj] = obj
    else:
        raise TypeError("save_under_key must be a callable function, or have a __setitem__ method.")
else:
    save_under_key(encoded_obj, obj)

# Grabbing frames

In [3]:
from mixing.video.video_files import save_frame

video = '/Users/thorwhalen/Downloads/cosmo_vids/_13a_Universe_Rearranges_in_Network_Animation.mp4'
save_frame(video, frame_idx=-1)

'/Users/thorwhalen/Downloads/cosmo_vids/_13a_Universe_Rearranges_in_Network_Animation_-00001.png'

In [4]:
# from clipboard (from copied filepath) to clipboard (copy image bytes)
from mixing.video.video_files import save_frame

save_frame(frame_idx=-1, copy_to_clipboard=True)

Getting video source from clipboard...
... Video source: /Users/thorwhalen/Downloads/i2i_vids/50b_Assembling_A_Prototype_Board.mp4
Copying image to clipboard...
Saved frame to: /Users/thorwhalen/Downloads/i2i_vids/50b_Assembling_A_Prototype_Board_-00001.png


'/Users/thorwhalen/Downloads/i2i_vids/50b_Assembling_A_Prototype_Board_-00001.png'

# Concatenating videos

In [1]:
import dol

s = dol.filesys.FileCollection('/Users/thorwhalen/Downloads/cosmo_vids/', max_levels=0)
s = dol.filt_iter(s, filt=lambda f: f.endswith('.mp4') and not f.startswith('/Users/thorwhalen/Downloads/cosmo_vids/_'))
video_files = sorted(s)
video_files

['/Users/thorwhalen/Downloads/cosmo_vids/01_Book_Skyscraper_City_Video.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/02_Book_Towers_Collapsing_Video.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/03d_Collapsing_Book_City_Cinematic_Video.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/04b_City_Collapse_Book_Page_Animation.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/05b_Collapsing_Book_Towers_Pages_in_Space.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/06a_Celestial_Book_Page_Eruption_Video.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/07b_page_to_stars_transition.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/10_Visualizing_Customer_Review_Concentration.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/11_Generating_Video_From_Image_Description.mp4',
 '/Users/thorwhalen/Downloads/cosmo_vids/12_Sphere_to_Constellation_Video_Generation.mp4']

In [2]:
from moviepy import VideoFileClip, concatenate_videoclips

# video_files ="""
# /Users/thorwhalen/Downloads/cosmo_vids/00_Book_Skyscraper_City_Video.mp4
# /Users/thorwhalen/Downloads/cosmo_vids/01_Book_Towers_Collapsing_Video.mp4
# /Users/thorwhalen/Downloads/cosmo_vids/03d_Collapsing_Book_City_Cinematic_Video.mp4
# """.split()


v = concatenate_videoclips(list(map(VideoFileClip, video_files)))
v.write_videofile("/Users/thorwhalen/Downloads/cosmo_vids/combined_20251014.mp4")

MoviePy - Building video /Users/thorwhalen/Downloads/cosmo_vids/combined_20251014.mp4.
MoviePy - Writing audio in combined_20251014TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video /Users/thorwhalen/Downloads/cosmo_vids/combined_20251014.mp4



MoviePy - Done !
MoviePy - video ready /Users/thorwhalen/Downloads/cosmo_vids/combined_20251014.mp4


# verify frame continuity

In [ ]:
from mixing.video.video_concat import verify_frame_continuity

match, diff = verify_frame_continuity(
    '/Users/thorwhalen/Downloads/cosmo_vids/00_Book_Skyscraper_City_Video.mp4',
    '/Users/thorwhalen/Downloads/cosmo_vids/01_Book_Towers_Collapsing_Video.mp4',
    save_comparison='/Users/thorwhalen/Downloads/cosmo_vids/frame_comparison.png'
)

# Frames-to-Video

In [12]:
from mixing.video.video_gen import generate_video

video_paths = generate_video(
    prompt="Camera rolls 90 degrees counter-clockwise, while flying papers glow so much till they look like stars.",
    first_frame='/Users/thorwhalen/Downloads/cosmo_vids/06a_Celestial_Book_Page_Eruption_Video.mp4',
    last_frame= '/Users/thorwhalen/Downloads/cosmo_vids/10_Visualizing_Customer_Review_Concentration.mp4',
    model="veo-2.0-generate-001",
    aspect_ratio="16:9",
    duration_seconds=8,
    save_video="/Users/thorwhalen/Downloads/",
)

# Handle both single path (string) and multiple paths (list)
if isinstance(video_paths, str):
    print(f"Generated video: {video_paths}")
elif isinstance(video_paths, list):
    print("Generated videos:")
    for i, p in enumerate(video_paths):
        print(f"  {i+1}: {p}")
else:
    print(f"Unexpected result type: {type(video_paths)} = {video_paths}")

Generated video: /Users/thorwhalen/Downloads/generated_video_ (3).mp4


# Concatenating videos

In [8]:
from mixing.video.video_concat import *

In [9]:
def trim_first_frame_from_subsequent_clips(clips: list[VideoFileClip]) -> Iterable[VideoFileClip]:
    """Keep first clip intact, trim first frame from subsequent clips."""
    yield clips[0]
    for clip in clips[1:]:
        yield clip.subclipped(1 / clip.fps)
        
def stitch_with_first_frame_trim(video_paths, output_path):
    """Trim first frame from subsequent clips (no transition)."""
    final_clip = concatenate_videos(
        video_paths,
        transform_clips=trim_first_frame_from_subsequent_clips,
        output_path=output_path
    )
    final_clip.close()
    print(f"First frame trim: Saved to {output_path}")


def stitch_with_crossfade(video_paths, output_path, *, duration=0.5):
    """Simple crossfade between clips."""
    final_clip = concatenate_videos(
        video_paths,
        transform_clips=lambda clips: crossfade_transition(clips, duration=duration),
        output_path=output_path
    )
    final_clip.close()
    print(f"Crossfade: Saved to {output_path}")


def stitch_with_trim_and_crossfade(video_paths, output_path, *, duration=0.4):
    """Trim first frame then crossfade."""
    final_clip = concatenate_videos(
        video_paths,
        transform_clips=lambda clips: trim_and_crossfade(clips, duration=duration),
        output_path=output_path
    )
    final_clip.close()
    print(f"Trim+Crossfade: Saved to {output_path}")


def stitch_with_black_fades(video_paths, output_path, *, duration=0.3):
    """Fade through black between clips."""
    final_clip = concatenate_videos(
        video_paths,
        transform_clips=lambda clips: fade_through_black(clips, duration=duration),
        output_path=output_path
    )
    final_clip.close()
    print(f"Black fades: Saved to {output_path}")


def stitch_with_overlap(video_paths, output_path, *, overlap=0.5):
    """Overlap and blend clips."""
    final_clip = concatenate_videos(
        video_paths,
        transform_clips=lambda clips: overlap_blend(clips, overlap=overlap),
        output_path=output_path
    )
    final_clip.close()
    print(f"Overlap blend: Saved to {output_path}")



video_paths = """
/Users/thorwhalen/Downloads/cosmo_vids/00_Book_Skyscraper_City_Video.mp4
/Users/thorwhalen/Downloads/cosmo_vids/01_Book_Towers_Collapsing_Video.mp4
/Users/thorwhalen/Downloads/cosmo_vids/02_meh_Catastrophic_Book_City_Collapse_Video.mp4
""".split()

# Execute all four options
base_path = "/Users/thorwhalen/Downloads/cosmo_vids"

# Option 1: Simple crossfade
stitch_with_crossfade(
    video_paths,
    f"{base_path}/concat_crossfade.mp4",
    duration=0.5
)

# Option 2: Trim first frame + crossfade (RECOMMENDED)
stitch_with_trim_and_crossfade(
    video_paths,
    f"{base_path}/concat_trim_crossfade.mp4",
    duration=0.4
)

# Option 3: Fade through black
stitch_with_black_fades(
    video_paths,
    f"{base_path}/concat_black_fades.mp4",
    duration=0.3
)

# Option 4: Overlap blend
stitch_with_overlap(
    video_paths,
    f"{base_path}/concat_overlap.mp4",
    overlap=0.5
)

print("\n✅ All four versions created! Compare them to see which you prefer.")

MoviePy - Building video /Users/thorwhalen/Downloads/cosmo_vids/concat_crossfade.mp4.
MoviePy - Writing audio in concat_crossfadeTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video /Users/thorwhalen/Downloads/cosmo_vids/concat_crossfade.mp4



MoviePy - Done !
MoviePy - video ready /Users/thorwhalen/Downloads/cosmo_vids/concat_crossfade.mp4
Crossfade: Saved to /Users/thorwhalen/Downloads/cosmo_vids/concat_crossfade.mp4
MoviePy - Building video /Users/thorwhalen/Downloads/cosmo_vids/concat_trim_crossfade.mp4.
MoviePy - Writing audio in concat_trim_crossfadeTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video /Users/thorwhalen/Downloads/cosmo_vids/concat_trim_crossfade.mp4



MoviePy - Done !
MoviePy - video ready /Users/thorwhalen/Downloads/cosmo_vids/concat_trim_crossfade.mp4
Trim+Crossfade: Saved to /Users/thorwhalen/Downloads/cosmo_vids/concat_trim_crossfade.mp4
MoviePy - Building video /Users/thorwhalen/Downloads/cosmo_vids/concat_black_fades.mp4.
MoviePy - Writing audio in concat_black_fadesTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video /Users/thorwhalen/Downloads/cosmo_vids/concat_black_fades.mp4



MoviePy - Done !
MoviePy - video ready /Users/thorwhalen/Downloads/cosmo_vids/concat_black_fades.mp4
Black fades: Saved to /Users/thorwhalen/Downloads/cosmo_vids/concat_black_fades.mp4
MoviePy - Building video /Users/thorwhalen/Downloads/cosmo_vids/concat_overlap.mp4.
MoviePy - Writing audio in concat_overlapTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
MoviePy - Writing video /Users/thorwhalen/Downloads/cosmo_vids/concat_overlap.mp4



MoviePy - Done !
MoviePy - video ready /Users/thorwhalen/Downloads/cosmo_vids/concat_overlap.mp4
Overlap blend: Saved to /Users/thorwhalen/Downloads/cosmo_vids/concat_overlap.mp4

✅ All four versions created! Compare them to see which you prefer.


In [ ]:
verify_frame_continuity(
    '/Users/thorwhalen/Downloads/cosmo_vids/00_Book_Skyscraper_City_Video.mp4',
    '/Users/thorwhalen/Downloads/cosmo_vids/01_Book_Towers_Collapsing_Video.mp4'
)

(False, 0.1358312724105846)

In [ ]:
match, diff = verify_frame_continuity(
    '/Users/thorwhalen/Downloads/cosmo_vids/00_Book_Skyscraper_City_Video.mp4',
    '/Users/thorwhalen/Downloads/cosmo_vids/01_Book_Towers_Collapsing_Video.mp4',
    save_comparison='/Users/thorwhalen/Downloads/cosmo_vids/frame_comparison.png'
)

Comparison saved to: /Users/thorwhalen/Downloads/cosmo_vids/frame_comparison.png
